In [1]:
import requests

In [2]:
KONG_HOST_IP = "localhost"
KONG_PORT = 8001

KEYCLOAK_URL = "http://localhost:8085/auth/"
USERNAME = "admin"
PASSWORD = 'Pa55w0rd'
MASTER_REALM_NAME = "master"

CLIENT_ID = "kong_test_client-4"


In [3]:
#def clean():
response = requests.get(f'http://{KONG_HOST_IP}:{KONG_PORT}/routes')
for _id in [e["id"] for e in response.json()["data"]]:
    requests.delete(f'http://{KONG_HOST_IP}:{KONG_PORT}/routes/{_id}')
response = requests.get(f'http://{KONG_HOST_IP}:{KONG_PORT}/services')
for _id in [e["id"] for e in response.json()["data"]]:
    requests.delete(f'http://{KONG_HOST_IP}:{KONG_PORT}/services/{_id}')

In [4]:
from keycloak import KeycloakAdmin



#%%

keycloak_admin = KeycloakAdmin(server_url=KEYCLOAK_URL,
                               username=USERNAME,
                           password=PASSWORD,
                               verify=True)

In [5]:
client_secret_key = "2b9bd5f6-3d73-45d8-8b27-9f76763a458d"
client_secret_key

'2b9bd5f6-3d73-45d8-8b27-9f76763a458d'

In [6]:
services = [
    {
        'name': "front_service",
        'url': f'http://localhost:8080/',
        'path': "front"

    },
        {
        'name': "api_service",
        'url': f'http://localhost/8005',
        'path': "api"

    }
]

REALM_NAME = "master"
KEYCLOAK_HOST_IP="keycloak"
KEYCLOAK_PORT=8080

introspection_url = f'http://{KEYCLOAK_HOST_IP}:{KEYCLOAK_PORT}/auth/realms/{REALM_NAME}/protocol/openid-connect/token/introspect'
discovery_url = f'http://{KEYCLOAK_HOST_IP}:{KEYCLOAK_PORT}/auth/realms/{REALM_NAME}/.well-known/openid-configuration'
"http://localhost:8080/auth/realms/master/protocol/openid-connect/token/introspect"
print(introspection_url)
print(discovery_url)

for service in services:
    data = service
    
    # Create Service
    response = requests.post(f'http://{KONG_HOST_IP}:{KONG_PORT}/services', data=data)
    created_service_id = requests.get(f'http://{KONG_HOST_IP}:{KONG_PORT}/services/'+data["name"]).json()["id"]
    
    # Create route
    data = {
        'service.id': f'{created_service_id}',
        'paths[]': f'/{service["path"]}',
    }

    response = requests.post(f'http://{KONG_HOST_IP}:{KONG_PORT}/services/{service["name"]}/routes', data=data)      
    # Configure OIDC 
    data = {
        'name': 'oidc',
        'config.client_id': f'{CLIENT_ID}',
        'config.client_secret': f'{client_secret_key}',
        'config.realm': f'{REALM_NAME}',
        'config.bearer_only': 'true',
        'config.introspection_endpoint':introspection_url,
        'config.discovery':discovery_url
    }

    response = requests.post(f'http://{KONG_HOST_IP}:{KONG_PORT}/services/{created_service_id}/plugins', data=data)

http://keycloak:8080/auth/realms/master/protocol/openid-connect/token/introspect
http://keycloak:8080/auth/realms/master/.well-known/openid-configuration


In [7]:
KONG_HOST_IP = "localhost"
KONG_PORT = 8001

services = [
    {
        'name': 'api_service',
        'url': 'http://localhost/8005',
        'path': "api"
    },
    {
        'name': "front_service",
        'url': f'http://localhost:8080/',
        'path': "front"
    }
]

SyntaxError: invalid syntax (<ipython-input-7-b9d80509aaef>, line 5)

In [9]:
for service in services:
    name = service["name"]
    if requests.get(f"http://{KONG_HOST_IP}:{KONG_PORT}/services/{name}").status_code == 404:
        requests.post(f"http://{KONG_HOST_IP}:{KONG_PORT}/services/",data = service)
        print(name, "created !")
    else: print(name, "already exists !")

front_service already exists !
api_service already exists !
